###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Defensa Nacional de Colombia

In [1]:
import pandas as pd

###Archivo gr35-i7pm.csv

Contiene información sobre minas intervenidas o intervención de unidades productoras mineras por explotación ilícita por parte de la Fuerza Pública.

In [2]:
df = pd.read_csv("https://www.datos.gov.co/resource/gr35-i7pm.csv?$limit=500000", dtype={'cod_muni': str})
df.head()

,fecha_hecho,cod_depto,departamento,cod_muni,municipio,clase_de_bien,cantidad,unidad_de_medida
0,2024-07-31T00:00:00.000,5,ANTIOQUIA,05113,BURITICA,NaN,5,NaN
1,2024-07-31T00:00:00.000,41,HUILA,41016,AIPE,NaN,1,NaN
2,2024-07-31T00:00:00.000,41,HUILA,41001,NEIVA,NaN,1,NaN
3,2024-07-31T00:00:00.000,41,HUILA,41524,PALERMO,NaN,1,NaN
4,2024-07-31T00:00:00.000,41,HUILA,41524,PALERMO,NaN,1,NaN


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11738 entries, 0 to 11737
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fecha_hecho       11738 non-null  object 
 1   cod_depto         11738 non-null  int64  
 2   departamento      11738 non-null  object 
 3   cod_muni          11738 non-null  object 
 4   municipio         11738 non-null  object 
 5   clase_de_bien     0 non-null      float64
 6   cantidad          11738 non-null  int64  
 7   unidad_de_medida  0 non-null      float64
dtypes: float64(2), int64(2), object(4)
memory usage: 733.8+ KB


### Eliminación de columnas irrelevantes para el proyecto

In [5]:
relevant_cols = ['cod_muni', 'departamento', 'municipio', 'fecha_hecho', 'cantidad']
intervened_mines_MinDefensa = df[relevant_cols]
intervened_mines_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11738 entries, 0 to 11737
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cod_muni      11738 non-null  object
 1   departamento  11738 non-null  object
 2   municipio     11738 non-null  object
 3   fecha_hecho   11738 non-null  object
 4   cantidad      11738 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 458.6+ KB


### Convertir columna fecha_hecho a tipo date

In [6]:
# Hacer una copia explícita del DataFrame
intervened_mines_MinDefensa_ = intervened_mines_MinDefensa.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime (sin especificar formato)
intervened_mines_MinDefensa_['fecha_hecho'] = pd.to_datetime(intervened_mines_MinDefensa_['fecha_hecho'], errors='coerce')

# Extraer solo el año
intervened_mines_MinDefensa_['fecha_hecho'] = intervened_mines_MinDefensa_['fecha_hecho'].dt.year

In [7]:
intervened_mines_MinDefensa_['fecha_hecho'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012, 2011, 2010], dtype=int32)

### Verificar valores nulos

In [8]:
intervened_mines_MinDefensa_.isnull().sum()

,0
cod_muni,0
departamento,0
municipio,0
fecha_hecho,0
cantidad,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [9]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'municipio']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(intervened_mines_MinDefensa_[column].unique())
    print()


Categorías en la columna 'departamento':
['ANTIOQUIA' 'HUILA' 'BOLÍVAR' 'BOYACÁ' 'VALLE DEL CAUCA' 'CHOCÓ'
 'CUNDINAMARCA' 'META' 'CAQUETÁ' 'PUTUMAYO' 'CAUCA' 'CALDAS' 'SANTANDER'
 'CÓRDOBA' 'NORTE DE SANTANDER' 'ATLÁNTICO' 'TOLIMA' 'CESAR' 'SUCRE'
 'NARIÑO' 'CASANARE' 'BOGOTÁ D.C.' 'MAGDALENA' 'QUINDÍO' 'RISARALDA'
 'LA GUAJIRA' 'GUAVIARE' 'BOYACA' 'BOLIVAR' 'AMAZONAS' 'BOGOTA D.C.'
 'CHOCO' 'ATLANTICO' 'CORDOBA' 'QUINDIO' 'CAQUETA' 'GUAINÍA' 'VAUPÉS'
 'ARAUCA']

Categorías en la columna 'municipio':
['BURITICA' 'AIPE' 'NEIVA' 'PALERMO' 'RIVERA' 'TURBACO'
 'SAN PABLO DE BORBUR' 'TUTA' 'ROLDANILLO' 'YOLOMBO' 'QUIBDO' 'RICAURTE'
 'VILLAVICENCIO' 'FLORENCIA' 'PUERTO GUZMAN' 'SANTANDER DE QUILICHAO'
 'ALEJANDRIA' 'SEGOVIA' 'SANTA CATALINA' 'LA CALERA' 'TULUA' 'HISPANIA'
 'SAMACA' 'RIOSUCIO' 'GARZON' 'GUAMAL' 'GIRON' 'PALMIRA' 'GIRALDO'
 'LORICA' 'SAHAGUN' 'TOCAIMA' 'REMEDIOS' 'SAN ROQUE' 'PAMPLONA'
 'SATIVANORTE' 'MANIZALES' 'PIAMONTE' 'PUERTO BOYACA' 'CACERES'
 'SABANAGRANDE' 'FALAN' 'CA

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [10]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [11]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    intervened_mines_MinDefensa_[col] = intervened_mines_MinDefensa_[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Verificar que los valores en "cod_muni" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [12]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "cod_muni" en el df intervened_mines_MinDefensa_

In [13]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
intervened_mines_MinDefensa_['cod_muni'] = intervened_mines_MinDefensa_['cod_muni'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = intervened_mines_MinDefensa_['cod_muni'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

True

In [14]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5]


In [15]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,cod_muni,count,percentage
0,5,11738,1.0


In [16]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = intervened_mines_MinDefensa_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_muni']])
    print()

Muestra de registros con longitud 5:
  cod_muni
0    05113
1    41016
2    41001
3    41524
4    41524



Nota: el 100%  de los codigos que identifican a los municipios en el df intervened_mines_MinDefensa corresponden a string de 5 caracteres, igual a los almacenados en la base de datos del proyecto

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [17]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [18]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [19]:
# Comparar listas de departamento
compare_lists(intervened_mines_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en intervened_mines_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en intervened_mines_MinDefensa_ que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [20]:
# Comparar listas de códigos
compare_lists(intervened_mines_MinDefensa_['cod_muni'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en intervened_mines_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en intervened_mines_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [21]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio']
final_intervened_mines_MinDefensa = intervened_mines_MinDefensa_.drop(columns=columns_to_drop)

In [22]:
# Adicionar columna para trazabilidad de la fuente
final_intervened_mines_MinDefensa['source_id'] = 36

In [23]:
final_intervened_mines_MinDefensa.columns

Index(['cod_muni', 'fecha_hecho', 'cantidad', 'source_id'], dtype='object')

In [24]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'cod_muni': 'dane_code',
    'fecha_hecho': 'year_of_incident',
    'cantidad': 'amount',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_intervened_mines_MinDefensa.rename(columns=translation_map, inplace=True)

In [26]:
#Estructura final del dataset a integrar a la base de datos
final_intervened_mines_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11738 entries, 0 to 11737
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   dane_code         11738 non-null  object
 1   year_of_incident  11738 non-null  int32 
 2   amount            11738 non-null  int64 
 3   source_id         11738 non-null  int64 
dtypes: int32(1), int64(2), object(1)
memory usage: 321.1+ KB


## Salvar en archivo csv en el drive

In [27]:
# Guardar en archivos CSV en el drive
final_intervened_mines_MinDefensa.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/intervened_mines_MinDefensa.csv', index=False)